# Setup stuff: Connectivity

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, DCxSfY1X7NDFUNMi9rgoA5HKqrFNF4NF9mFroRehMtPQ, 2>


#  Actual analytics ..

In [ ]:

#  View the graph
#

l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """ ,
   contextualize=True,
   )
l_result.view()


In [5]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


--


In [6]:

#  Getting the airport node id
#

#  l_airport = "MKE"
#  l_airport = "ORD"
#  l_airport = "DEN"
l_airport = "SJC"


l_query  = """
   MATCH (n: Airport) 
   WHERE n.airport_code = '{0}'
   RETURN ID(n) AS my_id
   """.format(l_airport)

l_airport_id = my_graph.query(l_query)["my_id"][0]

print("Airport id:  %s   %d" % (l_airport, l_airport_id))
print("")


          0/? [?op/s]

Airport id:  SJC   562949953421312



In [8]:

#  Running analytics.bfs() ..

#  From above
#
l_source_nodeid       = str(l_airport_id)


l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "bfs_" + str(l_cntr).zfill(4)


analytics.bfs(
   my_graph,
      #
   source               = l_source_nodeid,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

print("--")


          0/? [?op/s]

--


In [9]:

#  analytics.sssp() adds a new property to the node
#

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  l_result2 = my_graph.query(
#     """
#     MATCH (n) - [r] -> (m)
#     RETURN r
#     """ )
#  print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------+--------------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   |   n.bfs_0022 | n.type   |
|----+-----------------+-------------+-----------+------------------+------------------+--------------+----------|
|  0 |               1 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   |            1 | node     |
|  1 |               2 | ['Airport'] | Airport   | DEN              | Denver           |            1 | node     |
|  2 |               0 | ['Airport'] | Airport   | MKE              | Milwaukee        |            2 | node     |
|  3 | 562949953421312 | ['Airport'] | Airport   | SJC              | San Jose         |            0 | node     |
+----+-----------------+-------------+-----------+------------------+------------------+--------------+----------+


# Adding use of graph.project()

In [10]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph that some analytics will not like ..


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code     = 'SJC' AND m.restaurant_code  = 'PAP'
   CREATE (n) -[r: ATTRACTION { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


          0/? [?op/s]

          0/? [?op/s]

In [11]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )

print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )

print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+------------------+--------------+----------+---------------------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_code   | n.airport_name   |   n.bfs_0022 | n.type   | n.restaurant_code   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+------------------+--------------+----------+---------------------+-----------------------------|
|  0 |               0 | ['Airport']    | Airport   | MKE              | Milwaukee        |            2 | node     | nan                 | nan                         |
|  1 |               2 | ['Airport']    | Airport   | DEN              | Denver           |            1 | node     | nan                 | nan                         |
|  2 |               1 | ['Airport']    | Airport   | ORD              | Chicago O-Hare   |            1 | node     | nan                 | nan       

          0/? [?op/s]

+----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label    |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   | r.DRIVABLE   |
|----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------|
|  0 |               0 | ['Airport']    |               0 | FLIES_TO   |           66 |            1 | FLIES_TO |               1 | ['Airport']      | edge     | nan          |
|  1 |               2 | ['Airport']    |               3 | FLIES_TO   |          886 |            1 | FLIES_TO |               1 | ['Airport']      | edge     | nan          |
|  2 |               1 | ['Airport']    |               2 | FLIES_TO   |          886 |            1 | FLIES_TO |  

# Run sssp() again, both with and without project()

In [12]:

#  Running analytics.bfs() ..

#  From above
#
l_source_nodeid       = str(l_airport_id)


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "bfs_" + str(l_cntr).zfill(4)


analytics.bfs(
   my_graph,
      #
   source               = l_source_nodeid,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

print("--")


          0/? [?op/s]

--


In [13]:
#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     (nothing is output)


          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+------------------+--------------+--------------+----------+---------------------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_code   | n.airport_name   |   n.bfs_0022 |   n.bfs_0023 | n.type   | n.restaurant_code   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+------------------+--------------+--------------+----------+---------------------+-----------------------------|
|  0 |               1 | ['Airport']    | Airport   | ORD              | Chicago O-Hare   |            1 |            1 | node     | nan                 | nan                         |
|  1 |               0 | ['Airport']    | Airport   | MKE              | Milwaukee        |            2 |            2 | node     | nan                 | nan                         |
|  2 |               3 | ['Restaurant'] | nan       | nan              | na

In [14]:

#  Running analytics.bfs() should be run against a projection, a sub-graph ..

my_graph2 = my_graph.project(edge_types=["FLIES_TO"])


#  Running analytics.sssp() ..

#  From above
#
l_source_nodeid       = str(l_airport_id)


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "bfs_" + str(l_cntr).zfill(4)


analytics.bfs(
   my_graph2,
      #
   source               = l_source_nodeid,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

print("--")


          0/? [?op/s]

--


In [15]:

#  Check the output from routine above ..

l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )

print(tabulate(l_result, headers='keys', tablefmt='psql'))



          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+------------------+--------------+--------------+--------------+----------+---------------------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_code   | n.airport_name   |   n.bfs_0022 |   n.bfs_0023 |   n.bfs_0024 | n.type   | n.restaurant_code   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+------------------+--------------+--------------+--------------+----------+---------------------+-----------------------------|
|  0 |               1 | ['Airport']    | Airport   | ORD              | Chicago O-Hare   |            1 |            1 |            1 | node     | nan                 | nan                         |
|  1 |               2 | ['Airport']    | Airport   | DEN              | Denver           |            1 |            1 |            1 | node     | nan                 | nan                         |
